IMPORTING AND CALLING FUNCTIONS

In [14]:
%run mis_funciones.py

READING QUADRATURE COORDINATES FROM AN EXTERNAL FILE

In [ ]:
import numpy as np
# Let's read the provided file and load the coordinates into a NumPy array

#file_path_Qpoints = "coordenadas_uniformes.txt"

file_path_Qpoints = "coordenadas_cuasi_uniformes.txt"

# Load the coordinates from the file
Qpoints = np.loadtxt(file_path_Qpoints, delimiter=' ')

NN2 = Qpoints.shape[0]  

#print(f" Coordenadas de los puntos: {Nodos2}")


READING AND MODIFYING THE BOUNDARY MESH FILE

In [16]:
# Define the file paths
input_file_path =  'Capsula_malla3.mdpa'
output_file_path = 'Capsula_malla3.txt'

#input_file_path =  r'C:\Users\INFO 1\ARCHIVOS BEM-FEM\PROGRAMAS BEM\Programas_Julio_2020\Cube.mdpa'
#output_file_path = r'C:\Users\INFO 1\ARCHIVOS BEM-FEM\PROGRAMAS BEM\Programas_Julio_2020\Cube.txt'

# Read the input file
with open(input_file_path, 'r') as file:
    lines = file.readlines()

# Initialize variables to track sections
inside_elements = False
inside_elemental_data = False
modified_lines = []

# Process the input file
element_counter = 1
elemental_data_counter = 1

for line in lines:
    # Handle "Begin Elements Triangle2D3"
    if "Begin Elements Triangle2D3" in line:
        inside_elements = True
        modified_lines.append(line)  # Keep the section header
        continue
    elif "End Elements" in line and inside_elements:
        inside_elements = False
        modified_lines.append(line)  # Keep the section footer
        continue

    # Handle "Begin ElementalData cell_tags"
    if "Begin ElementalData cell_tags" in line:
        inside_elemental_data = True
        modified_lines.append(line)  # Keep the section header
        continue
    elif "End ElementalData cell_tags" in line and inside_elemental_data:
        inside_elemental_data = False
        modified_lines.append(line)  # Keep the section footer
        continue

    # Process lines inside "Begin Elements Triangle2D3"
    if inside_elements:
        parts = line.split()
        if len(parts) > 0:
            parts[0] = str(element_counter)  # Re-enumerate the first column
            element_counter += 1
        modified_lines.append(" ".join(parts) + "\n")
        continue

    # Process lines inside "Begin ElementalData cell_tags"
    if inside_elemental_data:
        parts = line.split()
        if len(parts) > 1 and parts[1] != "0":  # Remove rows with '0' in the second column
            parts[0] = str(elemental_data_counter)  # Re-enumerate the first column
            elemental_data_counter += 1
            modified_lines.append(" ".join(parts) + "\n")
        continue

    # For all other lines, keep them unchanged
    modified_lines.append(line)

# Write the modified lines to the output file
with open(output_file_path, 'w') as file:
    file.writelines(modified_lines)

output_file_path  # Return the path to the modified file


'Capsula_malla3.txt'

EXTRACTING VERTEX COORDINATES AND CONNECTIVITY

In [17]:
import numpy as np

# Initialize empty arrays
vertices = []
triangles= []

# Read a file
with open(output_file_path, 'r') as file:
    lines = file.readlines()

# Process sections
inside_nodes = False
inside_elements = False

for line in lines:
    # Detect sections
    if "Begin Nodes" in line:
        inside_nodes = True
        continue
    elif "End Nodes" in line:
        inside_nodes = False
        continue

    if "Begin Elements" in line:
        inside_elements = True
        continue
    elif "End Elements" in line:
        inside_elements = False
        continue

    # Store vertex coordinates
    if inside_nodes:
        parts = line.split()
        if len(parts) == 4:
            vertices.append([int(parts[0]), float(parts[1]), float(parts[2]), float(parts[3])])

    # Store elements, omitting the second column
    if inside_elements:
        parts = line.split()
        if len(parts) > 4:
            triangles.append([int(parts[0]), int(parts[2]), int(parts[3]), int(parts[4])])

# Convert the lists to NumPy arrays
vertices = np.array(vertices)
triangles=np.array(triangles)

# Restore NumPy print options to their default status
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=75, nanstr='nan', precision=8, suppress=False, threshold=1000, formatter=None)


print(vertices)
print(triangles)


[[ 1.00000000e+00 -2.00000000e+00 -2.44929360e-16  1.00000000e+00]
 [ 2.00000000e+00 -1.22464680e-16 -1.49975978e-32  3.00000000e+00]
 [ 3.00000000e+00 -2.00000000e+00 -2.44929360e-16 -1.00000000e+00]
 ...
 [ 3.35300000e+03 -1.56455786e+00  1.24585662e+00 -1.56138868e-01]
 [ 3.35400000e+03 -1.93147223e+00 -5.19052039e-01  1.38124996e-01]
 [ 3.35500000e+03 -1.93005657e+00 -5.24291542e-01  2.65150012e-01]]
[[   1  196  198  197]
 [   2  196  197   82]
 [   3  199  200   65]
 ...
 [6704 2633 2643 3322]
 [6705 2396 2412 2409]
 [6706 2881 2405 2682]]


SAVING VERTEX COORDINATES AND CONNECTIVTY

In [18]:

# File path where the vertices' coordinates will be saved
output_file_path_vertices = 'Coord_vertices.txt'
np.savetxt(output_file_path_vertices, vertices, fmt='%f', delimiter=' ')
print(f"The coordinates of the vertices were saved in: {output_file_path_vertices}")


# File path where the vertices' connectivity will be saved
output_file_path_elements = 'Element_connectivity.txt'
# Add the element index as the first column
num_element = triangles.shape[0]
indices = np.arange(1, num_element + 1).reshape(-1, 1)  
connectivity = np.hstack((indices, triangles))

np.savetxt(output_file_path_elements, connectivity, fmt='%d', delimiter=' ')
print(f"The connectivity of the vertices were saved in: {output_file_path_elements}")


The coordinates of the vertices were saved in: Coord_vertices.txt
The connectivity of the vertices were saved in: Element_connectivity.txt


INTEGRAL OF (f(x,y,z) = 3x² + y + z) COMPUTED BY APPLYING GREEN'S THEOREM AND ANALYTIC INTEGRATION OVER TRIANGLES (USED AS REFERENCE) 

In [19]:

vertices=vertices[:,1:4];
triangles=triangles[:,1:4];

min_index=np.min(triangles); #  


import numpy as np
np.set_printoptions(threshold=np.inf, linewidth=200)

# The integral is calcuated as the sum of analytic function evaluations values for all triangles

integral = 0;
u=triangles.shape[0];
display(u);
nx=np.zeros((u,3));
cont=0;
for tri in triangles:
    #display(tri)
    v1=vertices[tri[0]-min_index];
    v2=vertices[tri[1]-min_index];
    v3=vertices[tri[2]-min_index];
    #display('triangles')
    #display(v1)
    #display(v2)
    #display(v3)

    # Extracting vertex coordinates
    a1, b1, c1 = v1
    a2, b2, c2 = v2
    a3, b3, c3 = v3
    # Computing the unitary normal vector
    v12=v2-v1;
    v13=v3-v1;
    Vn=np.cross(v12,v13);
    Vn=Vn/np.linalg.norm(Vn);
    nx[cont]=Vn;

    val=funcion_analitica(a1, a2, a3, b1, b2, b3, c1, c2, c3);
    #display(valor)

    integral =integral +val ;

    cont=cont+1;
#display(nx)
print(f"The reference integral value is: {integral}")
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=75, nanstr='nan', precision=8, suppress=False, threshold=1000, formatter=None)



6706

The reference integral value is: 155.34041818217437


BUILDING THE AUGMENTED RADIAL BASIS FUNCTION  MATRIX

In [20]:
import numpy as np
from joblib import Parallel, delayed
import multiprocessing

# Load the coordinates from the file
# coordinates_file_path = "coordenadas_cuasi_uniformes.txt"
# coordinates= np.loadtxt(coordinates_file_path, delimiter=' ')

# NN2 = coordinates.shape[0]
# Qpoints = np.array(coordinates)

# Define the function to compute a block of the upper triangular matrix
def compute_upper_triangle_block(start_row, end_row, NN2, Qpoints):
    block = np.zeros((end_row - start_row, NN2))
    for ii in range(start_row, end_row):
        for jj in range(ii, NN2):
            Rij = np.linalg.norm(Qpoints[ii, :] - Qpoints[jj, :])
            value = (Rij**4) * np.log(Rij + np.finfo(float).eps)  # np.finfo(float).eps is a small constant
            block[ii - start_row, jj] = value
    return start_row, end_row, block

# Number of row blocks
num_blocks = multiprocessing.cpu_count()

# Calculate the block size
block_size = round((NN2 + num_blocks - 1) / num_blocks)-1;  # Ceiling division

print(f"Block size: {block_size}, NN2: {NN2}, num_blocks: {num_blocks}")

# Use joblib to compute blocks in parallel
C = np.zeros((NN2, NN2))
results = Parallel(n_jobs=num_blocks)(
    delayed(compute_upper_triangle_block)(i * block_size, min((i + 1) * block_size, NN2), NN2, Qpoints) for i in range(num_blocks)
)

# Assemble the matrix from the computed blocks
for start_row, end_row, block in results:
    C[start_row:end_row, start_row:NN2] = block[:, start_row:NN2]

# Fill the lower triangular part using symmetry
for i in range(NN2):
    for j in range(i):
        C[i, j] = C[j, i]


B = np.hstack([np.ones((NN2, 1)), Qpoints]) #
ZC = np.zeros((B.shape[1], B.shape[1]))  # 

AU = np.block([[C, B], [B.T, ZC]])  # 

#print("Matrix C:")
#print(C)
#print("\nMatrix AU:")
#print(AU)





Block size: 40, NN2: 798, num_blocks: 20


COMPUTING THE RBIM QUADRATURE INTEGRATION WEIGTHS

In [21]:
import numpy as np
from joblib import Parallel, delayed
import multiprocessing


# Computing the vector of Integrals 
NN = NN2
I_rbf = np.zeros((NN, 1))
xw = TriGaussPoints(6)
NP = xw.shape[0]

# Parallelizing the computation using joblib
num_cores = multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(calculate_I_rbf_for_node)(i, Qpoints, triangles, vertices, min_index, xw, NP) for i in range(NN))

# Assigning the results to the vector I_rbf
for i, value in results:
    I_rbf[i] = value

I_poly = np.zeros((4, 1))

for tri in triangles:
    v1 = vertices[tri[0] - min_index]
    v2 = vertices[tri[1] - min_index]
    v3 = vertices[tri[2] - min_index]
    a1, b1, c1 = v1
    a2, b2, c2 = v2
    a3, b3, c3 = v3
    J=(b1 * c2 - b1 * c3 - b2 * c1 + b2 * c3 + b3 * c1 - b3 * c2);
    Int1 = (a1 + a2 + a3) * J / 6
    Int2 = (a1**2 + a1*a2 + a1*a3 + a2**2 + a2*a3 + a3**2) * J / 24
    Int3 = J* (2 * a1 * b1 + a1 * b2 + a1 * b3 + a2 * b1 + 2 * a2 * b2 + a2 * b3 + a3 * b1 + a3 * b2 + 2 * a3 * b3) / 24
    Int4 = J * (2 * a1 * c1 + a1 * c2 + a1 * c3 + a2 * c1 + 2 * a2 * c2 + a2 * c3 + a3 * c1 + a3 * c2 + 2 * a3 * c3) / 24
    I_poly += np.array([[Int1], [Int2], [Int3], [Int4]])

#print(f"Integral rbf: {I_rbf}")
#print(f"Integral poly: {I_poly}")

IU = np.vstack((I_rbf, I_poly))  # Vector of Integrals

#print(f" Vector of Integrals: {IU}")


# Computing the RBIM quadrature weights
WI = np.dot(np.linalg.inv(AU), IU)  

WInt = WI[0:NN];




ESTIMATED ABSOLUTE ERROR

In [22]:
# Comparing the reference integral vs. the RBIM integral computed for the test function 3*x1^2 + x2 + x3

print(f" Reference value: {integral}")

x1 = Qpoints[:, 0]
x2 = Qpoints[:, 1]
x3 = Qpoints[:, 2]

f_test = 3 * x1**2 + x2 + x3  # Test function

I_quad = np.dot(WInt.T, f_test)
I_quad = I_quad[0]

print(f" RBIM computed value: {I_quad}")

error =np.abs( 100 * (integral - I_quad) / integral)
error=format(error,"0.5e")
print(f" The percentage of error obtained with {NN} integration points is: {error} %")

np.set_printoptions(edgeitems=3, infstr='inf', linewidth=75, nanstr='nan', precision=8, suppress=False, threshold=1000, formatter=None)


 Reference value: 155.34041818217437
 RBIM computed value: 155.342624601245
 The percentage of error obtained with 798 integration points is: 1.42038e-03 %


SAVING THE RBIM QUADRATURE

In [23]:
Quad=np.hstack([WInt, Qpoints])
np.savetxt("Quad_RBIM.txt", Quad, fmt='%f', delimiter=' ')